In [26]:
import mlflow
import mlflow.sklearn
import dagshub
import pandas as pd
import numpy as np
from utility import basic_data_cleaning
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer,TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, root_mean_squared_error,mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"E:\DATA SCIENCE & AI\PROJECTS\Swiggy Delivery Dataset\train.csv")

In [3]:
df.head(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [4]:
df = basic_data_cleaning(df)

In [5]:
df.head(5)

,Delivery_person_Age,Delivery_person_Ratings,Distance_res_loc_KM,order_dayname,time_of_day,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,37.0,4.9,3.025153,Saturday,morning,sunny,high,2,motorcycle,0.0,no,urban,24.0
1,34.0,4.5,20.183558,Friday,evening,stormy,jam,2,scooter,1.0,no,metropolitian,33.0
2,23.0,4.4,1.552760,Saturday,morning,sandstorms,low,0,motorcycle,1.0,no,urban,26.0
3,38.0,4.7,7.790412,Tuesday,evening,sunny,medium,0,motorcycle,1.0,no,metropolitian,21.0
4,32.0,4.6,6.210147,Saturday,afternoon,cloudy,high,1,scooter,1.0,no,metropolitian,30.0


In [6]:
df.shape

(45502, 13)

In [7]:
df.isnull().sum()

Delivery_person_Age        1854
Delivery_person_Ratings    1908
Distance_res_loc_KM        3630
order_dayname                 0
time_of_day                1640
Weatherconditions           525
Road_traffic_density        510
Vehicle_condition             0
Type_of_vehicle               0
multiple_deliveries         993
Festival                    228
City                       1198
Time_taken(min)               0
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

Delivery_person_Age        0
Delivery_person_Ratings    0
Distance_res_loc_KM        0
order_dayname              0
time_of_day                0
Weatherconditions          0
Road_traffic_density       0
Vehicle_condition          0
Type_of_vehicle            0
multiple_deliveries        0
Festival                   0
City                       0
Time_taken(min)            0
dtype: int64

In [10]:
df.shape

(38064, 13)

In [11]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [12]:
nom_cat_cols = ['order_dayname','time_of_day','Type_of_vehicle','Weatherconditions','Festival','City']
ord_cat_cols = ['Road_traffic_density']

In [13]:
trafic_list = ['low','medium','high','jam']

In [14]:
trasformer = ColumnTransformer(
            transformers=[
                ("OHE",OneHotEncoder(drop='first',sparse_output=False),nom_cat_cols),
                ("Ordinal",OrdinalEncoder(categories=[trafic_list],handle_unknown='use_encoded_value',unknown_value=-1,encoded_missing_value=-2),ord_cat_cols)
            ],remainder="passthrough"
        )

In [15]:
processor = Pipeline(
    steps=[
        ("First_transformation",trasformer),
        ("scaling",StandardScaler())
    ]
    )

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
X_train_trnsfmd = processor.fit_transform(X_train)
X_test_trnsfmd = processor.transform(X_test)

In [18]:
dagshub.init(repo_owner='Pravat-21', repo_name='Swiggy-Delivery-Time-Prediction', mlflow=True)

Accessing as Pravat-21

Initialized MLflow to track repo "Pravat-21/Swiggy-Delivery-Time-Prediction"

Repository Pravat-21/Swiggy-Delivery-Time-Prediction initialized!

In [19]:
pt = PowerTransformer()

In [20]:
mlflow.set_experiment("Exp 2-Baseline Model Selections")

<Experiment: artifact_location='mlflow-artifacts:/136bccc6397142babf68598b124532d3', creation_time=1765022577995, experiment_id='1', last_update_time=1765022577995, lifecycle_stage='active', name='Exp 2-Baseline Model Selections', tags={}>

In [45]:
def objective(trial):
    try:
        algo = trial.suggest_categorical("Algorithms",['LR','Lasso','Ridge','ElasticNet','SVR','KNN','RF','DTR','XGB','LGBM'])

        if algo == 'LR':
            model = TransformedTargetRegressor(regressor=LinearRegression(),transformer=pt)

        elif algo == 'Lasso':
            alpha = trial.suggest_float("Alpha",0.5,3.0)
            selection = trial.suggest_categorical("selection",['cyclic','random'])

            reg = Lasso(alpha=alpha,selection=selection,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'Ridge':
            alpha = trial.suggest_float("Alpha",0.5,3.0)
            solver = trial.suggest_categorical("solver",['auto','svd'])

            reg = Ridge(alpha=alpha,solver=solver,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'ElasticNet':
            alpha = trial.suggest_float("Alpha",0.5,3.0)
            selection = trial.suggest_categorical("selection",['cyclic','random'])

            reg = ElasticNet(alpha=alpha,selection=selection,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'SVR':
            kernel = trial.suggest_categorical('kernel',['linear','rbf','sigmoid'])
            gamma = trial.suggest_categorical("gamma",['scale','auto'])
            C = trial.suggest_float("C",1.0,3.0)

            reg = SVR(kernel=kernel,gamma=gamma,C=C)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'KNN':
            n_neighbors = trial.suggest_int("n_neighbors",3,10)
            weights = trial.suggest_categorical("weights",['uniform','distance'])
            algorithm = trial.suggest_categorical("algorithm",['auto','ball_tree','kd_tree'])

            reg = KNeighborsRegressor(n_neighbors=n_neighbors,weights=weights,algorithm=algorithm)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'RF':
            n_estimators = trial.suggest_int('n_estimators',100,800)
            max_depth = trial.suggest_int('max_depth',5,30)

            reg = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'DTR':
            max_depth = trial.suggest_int('max_depth',5,30)
            splitter =trial.suggest_categorical('splitter',['best','random'])

            reg = DecisionTreeRegressor(max_depth=max_depth,splitter=splitter,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        elif algo == 'XGB':
            n_estimators = trial.suggest_int('n_estimators',100,800)
            max_depth = trial.suggest_int('max_depth',5,30)
            max_leaves = trial.suggest_int('max_leaves',5,30)
            learning_rate = trial.suggest_float('learning_rate',1e-3,1e-1,log =True)

            reg = XGBRegressor(n_estimators=n_estimators,max_depth=max_depth,max_leaves=max_leaves,learning_rate=learning_rate)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)
    
        elif algo == 'LGBM':
            n_estimators = trial.suggest_int('n_estimators',100,800)
            max_depth = trial.suggest_int('max_depth',5,30)
            num_leaves = trial.suggest_int('num_leaves',5,30)
            learning_rate = trial.suggest_float('learning_rate',1e-3,1e-1,log =True)

            reg = LGBMRegressor(n_estimators=n_estimators,max_depth=max_depth,num_leaves=num_leaves,learning_rate=learning_rate,random_state=42)
            model = TransformedTargetRegressor(regressor=reg,transformer=pt)

        run_name = f"Trial-{trial.number}:Algorithm-{algo}"

        with mlflow.start_run(run_name=run_name,nested=True) as child_run:

            mlflow.set_tags({
                "Model-name":f"{algo}",
                "Experiment-type":"Baseline Model Selection",
                "author-name":"Pravat",
                "Description":f"This is the result of experiment no {trial.number}."
            })

            model.fit(X_train_trnsfmd,y_train)

            y_pred_train = model.predict(X_train_trnsfmd)
            y_pred = model.predict(X_test_trnsfmd)

            r2score_test = r2_score(y_test,y_pred)
            r2score_train = r2_score(y_train,y_pred_train)

            mae = mean_absolute_error(y_test,y_pred)
            mae_train = mean_absolute_error(y_train,y_pred_train)

            rmse = root_mean_squared_error(y_test,y_pred)
            rmse_train = root_mean_squared_error(y_train,y_pred_train)

            mlflow.log_metrics({
                "training_R2_score":r2score_train,
                "testing_R2_score":r2score_test,
                "training_MAE":mae_train,
                "testing_mae":mae,
                "training_rmse":rmse_train,
                "testing_rmse":rmse
            })
        
            mlflow.log_param("model",f"{algo}")
            mlflow.log_params(model.get_params())


            print({
                "training_R2_score":r2score_train,
                "testing_R2_score":r2score_test,
                "training_MAE":mae_train,
                "testing_mae":mae,
                "training_rmse":rmse_train,
                "testing_rmse":rmse
            })

            return rmse
    
    except Exception as e:
        print(f"Skipping Trial {trial.number}. Reason: {e}")
        raise optuna.TrialPruned()

In [46]:
with mlflow.start_run(run_name="Best Algo and Parameters with rmse") as parent_run:

    
    
    # Run Optuna optimization
    study = optuna.create_study(direction="minimize",study_name="Baseline Model Selection")
    study.optimize(objective, n_trials=50)

    # Log BEST trial metrics in the parent run
    mlflow.set_tags(
        {
            "Best-Model-name":f"{study.best_trial.params['Algorithms']}",
            "Experiment-type":"Baseline Model Selection",
            "author-name":"Pravat",
            "Description":f"This is the result of experiment no {study.best_trial}."
        }
    )
    dict1 = study.best_params
    algo = dict1.pop('Algorithms')
    

    if algo == 'LR':
        reg = LinearRegression(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'Lasso':
        reg = Lasso(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'Ridge':
        reg = Ridge(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'ElasticNet':
        reg = ElasticNet(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'SVR':
        reg = SVR(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'KNN':
        reg = KNeighborsRegressor(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'RF':
        reg = RandomForestRegressor(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'DTR':
        reg = DecisionTreeRegressor(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)

    elif algo == 'XGB':
        reg = XGBRegressor(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)
    
    elif algo == 'LGBM':
        reg = LGBMRegressor(**dict1)
        model = TransformedTargetRegressor(regressor=reg,transformer=pt)


    model.fit(X_train_trnsfmd,y_train)

    y_pred_train = model.predict(X_train_trnsfmd)
    y_pred = model.predict(X_test_trnsfmd)

    r2score_test = r2_score(y_test,y_pred)
    r2score_train = r2_score(y_train,y_pred_train)

    mae = mean_absolute_error(y_test,y_pred)
    mae_train = mean_absolute_error(y_train,y_pred_train)

    rmse = root_mean_squared_error(y_test,y_pred)
    rmse_train = root_mean_squared_error(y_train,y_pred_train)

    mlflow.log_metrics({
        "best_training_R2_score":r2score_train,
        "best_testing_R2_score":r2score_test,
        "best_training_MAE":mae_train,
        "best_testing_mae":mae,
        "best_training_rmse":rmse_train,
        "best_testing_rmse":rmse
    })
    print({
        "training_R2_score":r2score_train,
        "testing_R2_score":r2score_test,
        "training_MAE":mae_train,
        "testing_mae":mae,
        "training_rmse":rmse_train,
        "testing_rmse":rmse
    })
    mlflow.log_params(study.best_trial.params)

    print("Optimization completed!")
    print("Best R2:", study.best_trial.value)
    print("Best Params:", study.best_trial.params)

[I 2025-12-06 22:51:40,675] A new study created in memory with name: Baseline Model Selection


{'training_R2_score': -0.013393565133749341, 'testing_R2_score': -0.009808124016403186, 'training_MAE': 7.511640835110512, 'testing_mae': 7.561802582037067, 'training_rmse': 9.387151602571182, 'testing_rmse': 9.397529841424046}
🏃 View run Trial-0:Algorithm-Lasso at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1/runs/e18217083aeb4d67923a52599c6769bb
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1


[I 2025-12-06 22:51:43,220] Trial 0 finished with value: 9.397529841424046 and parameters: {'Algorithms': 'Lasso', 'Alpha': 2.0734884143063614, 'selection': 'cyclic'}. Best is trial 0 with value: 9.397529841424046.


{'training_R2_score': 0.5951893530921574, 'testing_R2_score': 0.5920752104141402, 'training_MAE': 4.7157879624322625, 'testing_mae': 4.769732638677736, 'training_rmse': 5.932950484776277, 'testing_rmse': 5.972887466530514}
🏃 View run Trial-1:Algorithm-LR at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1/runs/b642ec2eb5c54f3f9c2bd49b69769ce5
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1


[I 2025-12-06 22:51:45,655] Trial 1 finished with value: 5.972887466530514 and parameters: {'Algorithms': 'LR'}. Best is trial 1 with value: 5.972887466530514.


...

{'training_R2_score': 0.5951886196401917, 'testing_R2_score': 0.5920757330111399, 'training_MAE': 4.715788993383538, 'testing_mae': 4.769724311245728, 'training_rmse': 5.932955859551196, 'testing_rmse': 5.972883640562897}
🏃 View run Trial-48:Algorithm-Ridge at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1/runs/5bc646c5b76a4eeda939f57de0b52dd6
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1


[I 2025-12-06 23:23:54,685] Trial 48 finished with value: 5.972883640562897 and parameters: {'Algorithms': 'Ridge', 'Alpha': 1.1203880293370427, 'solver': 'auto'}. Best is trial 25 with value: 3.845442378853777.


{'training_R2_score': 0.5951893530921574, 'testing_R2_score': 0.5920752104141402, 'training_MAE': 4.7157879624322625, 'testing_mae': 4.769732638677736, 'training_rmse': 5.932950484776277, 'testing_rmse': 5.972887466530514}
🏃 View run Trial-49:Algorithm-LR at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1/runs/bb611f90126b4412a035d0b1b16a47f4
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1


[I 2025-12-06 23:24:01,689] Trial 49 finished with value: 5.972887466530514 and parameters: {'Algorithms': 'LR'}. Best is trial 25 with value: 3.845442378853777.


{'training_R2_score': 0.9216453251841785, 'testing_R2_score': 0.831151688565921, 'training_MAE': 2.0423638760578826, 'testing_mae': 3.0951750090105192, 'training_rmse': 2.6102206246656845, 'testing_rmse': 3.8427545390776983}
Optimization completed!
Best R2: 3.845442378853777
Best Params: {'Algorithms': 'RF', 'n_estimators': 494, 'max_depth': 15}
🏃 View run Best Algo and Parameters with rmse at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1/runs/2a1a2954f4984849ae39264e6744e138
🧪 View experiment at: https://dagshub.com/Pravat-21/Swiggy-Delivery-Time-Prediction.mlflow/#/experiments/1


In [47]:
hp_df = study.trials_dataframe()

In [48]:
hp_df['params_Algorithms'].value_counts()

params_Algorithms
XGB           12
RF            10
SVR            6
LR             4
Ridge          4
Lasso          3
LGBM           3
ElasticNet     3
KNN            3
DTR            2
Name: count, dtype: int64

In [49]:
hp_df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_Algorithms', 'params_Alpha', 'params_C', 'params_algorithm',
       'params_gamma', 'params_kernel', 'params_learning_rate',
       'params_max_depth', 'params_max_leaves', 'params_n_estimators',
       'params_n_neighbors', 'params_num_leaves', 'params_selection',
       'params_solver', 'params_splitter', 'params_weights', 'state'],
      dtype='object')

In [52]:
round(study.trials_dataframe().groupby("params_Algorithms")['value'].mean().sort_values(),2)

params_Algorithms
RF                3.92
XGB               4.03
DTR               5.44
KNN               5.50
LGBM              5.65
Ridge             5.97
LR                5.97
ElasticNet        9.40
Lasso             9.40
SVR           57724.45
Name: value, dtype: float64

**So, we will select `RandomForest` & `LGBMRegressor` for creating our stacking estimator.**